# Walmart Sales Prediction

In [1]:
import pandas as pd

events_holidays_df = pd.read_excel('../data/Events_HolidaysData.xlsx')
macro_economic_df = pd.read_excel('../data//macro_economic.xlsx')
weather_df = pd.read_excel('../data/WeatherData.xlsx')
train_df = pd.read_csv('../data/train.csv')
submission = pd.read_csv('../data/submission.csv')
kaggle_submission = pd.read_csv('../data/Kaggle_Submission_Format.csv')
attributes_description_df = pd.read_excel('../data/attributes_description.xlsx')

In [2]:
events_holidays_df.head()

,Year,MonthDate,Event,DayCategory
0,2009,2001-01-01,New Year's Day,Federal Holiday
1,2009,2019-01-01,Martin Luther King Jr. Day,Federal Holiday
2,2009,2014-02-01,Valentine's Day,Event
3,2009,2016-02-01,Presidents' Day,Federal Holiday
4,2009,2012-04-01,Easter Sunday,Event


In [3]:
print(events_holidays_df.columns)

Index(['Year', 'MonthDate', 'Event', 'DayCategory'], dtype='object')


In [4]:
macro_economic_df.head()

,Year-Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,PartyInPower,unemployment rate,CommercialBankInterestRateonCreditCardPlans,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Earnings or wages in dollars per hour,AdvertisingExpenses (in Thousand Dollars),Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
0,2009 - Jan,14421.752895,14407.053343,233.402,Democrats,7.8,12.03,11.44,22.05,137,57.70,4.02,9.296,7.559,799,12.589,4.17,11.550
1,2009 - Feb,14389.200466,14366.176571,234.663,Democrats,8.3,12.97,11.05,22.22,200,55.21,-4.32,9.296,7.559,799,12.589,3.87,11.100
2,2009 - Mar,14340.701639,14351.786822,235.067,Democrats,8.7,12.97,11.05,22.22,?,51.50,-6.72,9.296,7.559,799,12.589,3.72,11.650
3,2009 - Apr,14326.815525,14351.601731,235.582,Democrats,9.0,12.97,11.05,22.13,214,56.78,10.25,9.296,7.559,787,12.400,3.62,12.225
4,2009 - May,14345.904809,14368.123959,235.975,Democrats,9.4,13.32,11.25,22.04,?,61.95,9.11,9.297,7.400,803,12.384,3.52,12.300


In [5]:
print(macro_economic_df.columns)

Index(['Year-Month', 'Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI', 'PartyInPower',
       'unemployment rate', 'CommercialBankInterestRateonCreditCardPlans',
       'Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan',
       'Earnings or wages  in dollars per hour',
       'AdvertisingExpenses (in Thousand Dollars)',
       'Cotton Monthly Price - US cents per Pound(lbs)', 'Change(in%)',
       'Average upland planted(million acres)',
       'Average upland harvested(million acres)', 'yieldperharvested acre',
       'Production (in  480-lb netweright in million bales)',
       'Mill use  (in  480-lb netweright in million bales)', 'Exports'],
      dtype='object')


In [6]:
weather_df.head()

,Year,Month,Day,Temp high (°C),Temp avg (°C),Temp low (°C),Dew Point high (°C),Dew Point avg (°C),Dew Point low (°C),Humidity (%) high,...,Sea Level Press. (hPa) avg,Sea Level Press. (hPa) low,Visibility (km) high,Visibility (km) avg,Visibility (km) low,Wind (km/h) low,Wind (km/h) avg,Wind (km/h) high,Precip. (mm) sum,WeatherEvent
0,2009,Jan,1,-3,-6,-9,-16,-17,-19,54,...,1023,1015,16,16,16,37,18,60,0,NaN
1,2009,Jan,2,1,-2,-5,-3,-7,-17,78,...,1012,1007,16,13,2,27,10,48,T,Snow
2,2009,Jan,3,3,1,-2,-5,-9,-13,72,...,1015,1008,16,16,16,27,16,42,T,NaN
3,2009,Jan,4,6,1,-4,-10,-12,-13,55,...,1017,1015,16,16,16,32,12,40,0,NaN
4,2009,Jan,5,6,5,3,-1,-5,-16,62,...,1014,1013,16,16,16,23,11,34,T,NaN


In [7]:
print(weather_df.columns)

Index(['Year', 'Month', 'Day', 'Temp high (°C)', 'Temp avg (°C)',
       'Temp low (°C)', 'Dew Point high (°C)', 'Dew Point avg (°C)',
       'Dew Point low (°C)', 'Humidity (%) high', 'Humidity (%) avg',
       'Humidity (%) low', 'Sea Level Press. (hPa) high',
       'Sea Level Press. (hPa) avg', 'Sea Level Press. (hPa) low',
       'Visibility (km) high', 'Visibility (km) avg', 'Visibility (km) low',
       'Wind (km/h) low', 'Wind (km/h) avg', 'Wind (km/h) high',
       'Precip. (mm) sum', 'WeatherEvent'],
      dtype='object')


In [8]:
train_df.head()

,Year,Month,ProductCategory,Sales(In ThousandDollars)
0,2009,1,WomenClothing,1755.0
1,2009,1,MenClothing,524.0
2,2009,1,OtherClothing,936.0
3,2009,2,WomenClothing,1729.0
4,2009,2,MenClothing,496.0


In [9]:
print(train_df.columns)

Index(['Year', 'Month', 'ProductCategory', 'Sales(In ThousandDollars)'], dtype='object')


In [10]:
submission.head()

,Year,Month,ProductCategory,Unnamed: 3,Sales(In ThousandDollars)
0,2014,1,WomenClothing,NaN,NaN
1,2014,1,MenClothing,NaN,NaN
2,2014,1,OtherClothing,NaN,NaN
3,2014,2,WomenClothing,NaN,NaN
4,2014,2,MenClothing,NaN,NaN


In [11]:
print(submission.columns)

Index(['Year', 'Month', 'ProductCategory', 'Unnamed: 3',
       'Sales(In ThousandDollars)'],
      dtype='object')


In [12]:
kaggle_submission.head()

,Year,Sales(In ThousandDollars)
0,1,1000
1,2,1000
2,3,1000
3,4,1000
4,5,1000


In [13]:
print(kaggle_submission.columns)

Index(['Year', 'Sales(In ThousandDollars)'], dtype='object')


In [14]:
attributes_description_df.head()

,AttributeName,Type,Description
0,Year,temporal,Year
1,Month,temporal,Month
2,ProductCategory,categorical,"Clothing Products category - WomenClothing, Me..."
3,Sales(In ThousandDollars),numeric,Value of the sales or revenue in thousand dollars


In [15]:
print(len(attributes_description_df))

4


In [16]:
print(events_holidays_df.isnull().sum())

Year           0
MonthDate      0
Event          0
DayCategory    0
dtype: int64


In [17]:
print(macro_economic_df.isnull().sum())

Year-Month                                                           0
Monthly Nominal GDP Index (inMillion$)                               0
Monthly Real GDP Index (inMillion$)                                  0
CPI                                                                  0
PartyInPower                                                         0
unemployment rate                                                    0
CommercialBankInterestRateonCreditCardPlans                          0
Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan    0
Earnings or wages  in dollars per hour                               0
AdvertisingExpenses (in Thousand Dollars)                            0
Cotton Monthly Price - US cents per Pound(lbs)                       0
Change(in%)                                                          0
Average upland planted(million acres)                                0
Average upland harvested(million acres)                              0
yieldp

In [18]:
print(weather_df.isnull().sum())

Year                             0
Month                            0
Day                              0
Temp high (°C)                   0
Temp avg (°C)                    0
Temp low (°C)                    0
Dew Point high (°C)              0
Dew Point avg (°C)               0
Dew Point low (°C)               0
Humidity (%) high                0
Humidity (%) avg                 0
Humidity (%) low                 0
Sea Level Press. (hPa) high      0
Sea Level Press. (hPa) avg       0
Sea Level Press. (hPa) low       0
Visibility (km) high             0
Visibility (km) avg              0
Visibility (km) low              0
Wind (km/h) low                  0
Wind (km/h) avg                  0
Wind (km/h) high                 0
Precip. (mm) sum                 0
WeatherEvent                   225
dtype: int64


In [19]:
print(train_df.isnull().sum())

Year                          0
Month                         0
ProductCategory               0
Sales(In ThousandDollars)    10
dtype: int64


In [20]:
print(submission.isnull().sum())

Year                          0
Month                         0
ProductCategory               0
Unnamed: 3                   36
Sales(In ThousandDollars)    36
dtype: int64


In [21]:
print(kaggle_submission.isnull().sum())

Year                         0
Sales(In ThousandDollars)    0
dtype: int64


In [22]:
weather_df.fillna(method='bfill', inplace=True)

C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_12444\1073501904.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  weather_df.fillna(method='bfill', inplace=True)


In [23]:
print(weather_df.isnull().sum())

Year                           0
Month                          0
Day                            0
Temp high (°C)                 0
Temp avg (°C)                  0
Temp low (°C)                  0
Dew Point high (°C)            0
Dew Point avg (°C)             0
Dew Point low (°C)             0
Humidity (%) high              0
Humidity (%) avg               0
Humidity (%) low               0
Sea Level Press. (hPa) high    0
Sea Level Press. (hPa) avg     0
Sea Level Press. (hPa) low     0
Visibility (km) high           0
Visibility (km) avg            0
Visibility (km) low            0
Wind (km/h) low                0
Wind (km/h) avg                0
Wind (km/h) high               0
Precip. (mm) sum               0
WeatherEvent                   0
dtype: int64


In [25]:
train_df['Sales(In ThousandDollars)'].fillna(train_df['Sales(In ThousandDollars)'].mean(), inplace=True)


C:\Users\abraham.shimelis\AppData\Local\Temp\ipykernel_12444\1224873921.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Sales(In ThousandDollars)'].fillna(train_df['Sales(In ThousandDollars)'].mean(), inplace=True)


In [26]:
print(train_df.isnull().sum())

Year                         0
Month                        0
ProductCategory              0
Sales(In ThousandDollars)    0
dtype: int64
